In [1]:
!pip install --upgrade cohere
!pip install python-dotenv
!pip install --target=/usr/local/lib/python3.10/dist-packages annoy
!pip show annoy

import sys
sys.path.append('my_lib')
print(sys.path)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 92.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are instal

In [ ]:
# !echo 'y' | pip uninstall annoy


In [2]:
import cohere
from annoy import AnnoyIndex
import numpy as np
import pandas as pd
import warnings
import re
from dotenv import load_dotenv

In [5]:
# create text files from specified web pages and upload them here for colab
# also upload your dotenv file for your api key
from google.colab import files
uploaded = files.upload()
from google.colab import drive
drive.mount('/content/drive')
# access files with:
# '/content/drive/MyDrive/<your folder inside drive>/file.ext'

Saving UBCBTM_careers.txt to UBCBTM_careers (1).txt
Saving UBCBTM_reqs.txt to UBCBTM_reqs (1).txt
Saving BCOM_General.txt to BCOM_General (1).txt
Mounted at /content/drive


In [6]:
# also upload your dotenv file for your api key
import os
load_dotenv('.env')
COHERE_API_KEY = os.getenv('COHERE_API_KEY')
# make a .env file in the following format:
# COHERE_API_KEY="\insert\your\api\key"

In [ ]:
# def clean_page(file_name, title):
#   with open(file_name, 'r') as file:
#     text = file.read() + "\n\n"
#   text = re.split(r'\n\n|\n \n', text)
#   text = list(filter(' '.__ne__, text))
#   for i, par in enumerate(text):
#     text[i] = f"{title}: " + par
#   return text

In [7]:
def clean_page(file_name, title):
  with open(file_name, 'r') as file:
    text = file.read() + "\n\n"
  text = f"{title}: {text}".strip()
  return [text]

In [8]:
db = []

In [9]:
db += clean_page('BCOM_General.txt', "BCOM Program General Requirements")
db += clean_page('UBCBTM_reqs.txt', "Business Technology Management requirements")
db += clean_page('UBCBTM_careers.txt', "Business Technology Management careers")

In [ ]:
# db = np.array(db)

In [10]:
db[0]

"BCOM Program General Requirements: Bachelor of Commerce \nDegree requirements for year 1 entry in 2022W or later\nWe encourage you to make use of the BCom course planning worksheets as much as possible for the duration of your degree. We also encourage you to consult with academic advisors to develop a comprehensive academic plan that aligns with your academic goals and ensures timely progression towards degree completion, particularly if you are considering a reduced course load. Academic advisors can offer valuable insights and assistance in selecting courses, managing course loads, and navigating potential challenges in course registration. To book an appointment with an Advisor, please go here: https://calendly.com/bcom-ugo-advising\n\nContinuation in the BCom program \n\nA winter session average of ≥60% based on all courses attempted from September to April is required to continue in the program at all year levels. For more information, please see our Academic Standing page. \n\n

In [11]:
co = cohere.Client("GEuNRU8vOnitOMlJM2G3sXJqjMkLgvQyxy3uMjfk")

In [12]:
embeds = co.embed(
    texts=db
).embeddings

In [13]:
embeds = np.array(embeds)

In [14]:
print(embeds.shape)

(3, 4096)


In [15]:
search_index = AnnoyIndex(embeds.shape[1], 'angular')
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])
search_index.build(10) # 10 trees

True

In [16]:
def search_pages(query):
  query_embed = co.embed(texts=[query]).embeddings

  similar_page_ids = search_index.get_nns_by_vector(query_embed[0], 10, include_distances=True)
  return db[similar_page_ids[0]]

In [17]:
db = np.array(db)

In [18]:
results = search_pages(
    "do I need to take comm 296 to get a BUCS degree?"
)

In [19]:
results[0]

"BCOM Program General Requirements: Bachelor of Commerce \nDegree requirements for year 1 entry in 2022W or later\nWe encourage you to make use of the BCom course planning worksheets as much as possible for the duration of your degree. We also encourage you to consult with academic advisors to develop a comprehensive academic plan that aligns with your academic goals and ensures timely progression towards degree completion, particularly if you are considering a reduced course load. Academic advisors can offer valuable insights and assistance in selecting courses, managing course loads, and navigating potential challenges in course registration. To book an appointment with an Advisor, please go here: https://calendly.com/bcom-ugo-advising\n\nContinuation in the BCom program \n\nA winter session average of ≥60% based on all courses attempted from September to April is required to continue in the program at all year levels. For more information, please see our Academic Standing page. \n\n

In [20]:
def ask(question, num_generations=1):
  results = search_pages(question)
  context = results[0]

  prompt = f"""
  Extract the answer of the following question from the text provided.

  Question: {question}

  Provided text from the web page titled \"{context[:context.find(":")]}\": {context}
  """

  prediction = co.generate(
      prompt=prompt,
      max_tokens=200,
      model="command-nightly",
      temperature=0.5,
      num_generations=num_generations
  )

  return prediction.generations

In [ ]:
def ask_debug(question, num_generations=1):
  results = search_pages(question)
  context = results[0]

  prompt = f"""
  Excerpt from the web page titled \"{context[:context.find(":")]}\"

  Extract the answer of the following question from the text provdided.
  If the text doesn't contain the answer, reply that the answer is not available.

  Question: {question}
  """

  prediction = co.generate(
      prompt=prompt,
      max_tokens=200,
      model="command-nightly",
      temperature=0.5,
      num_generations=num_generations
  )

  return prediction.generations

In [22]:
results = ask("List me courses that need to be completed before Year 3 for the BCOM degree", num_generations=5)
for i, res in enumerate(results):
  print(f"result {i}: \n\n{res}")

result 0: 

id='f1e6389d-0ecd-48b6-82ea-4af4f97bce89' text='The answer is The following courses need to be completed before Year 3 for the BCOM degree: \n\nClass Standing 1 / First Year: \n- ECON 101 - Principles of Microeconomics\n- ECON 102 - Principles of Macroeconomics\n- COMM 101 - Business Fundamentals\n- COMM 105 - Values, Ethics, and Community\n- COMM 190 - Introduction to Quantitative Decision Making\n- COMM 191 - Application of Statistics in Business\n- COMM 192 - Management and Organizational Behaviour\n- COMM 196 - Business Communications I\n- MATH 100 or equivalent - Differential Calculus with Applications\n- Non-Commerce electives \n\nClass Standing 2 / Second Year: \n- COMM 202 - Career Fundamentals\n- COMM 203 - Managing the Employment Relationship\n- COMM 204 - Logistics and Operations Management\n- COMM' index=None likelihood=None token_likelihoods=None
result 1: 

id='b52774f3-25b7-45c0-9a99-fb797fa91e7f' text='The answer is The following courses need to be completed